In [1]:
from aiida import load_profile
from aiida.orm import Int
from aiida_workgraph import task, WorkGraph

In [2]:
from pickle import loads

In [3]:
@task.calcfunction(outputs=[{"name": "x"}, {"name": "y"}, {"name": "z"}])
def add_x_and_y(x, y):
    z = x + y
    return {"x": x, "y": y, "z": z}

In [4]:
@task.calcfunction()
def add_x_and_y_and_z(x, y, z):
    w = x + y + z
    return w

In [5]:
load_profile()

Profile<uuid='7bb8761123324468bb98821cbb757251' name='presto'>

In [6]:
wg = WorkGraph("my_workflow")
wg.add_task(add_x_and_y, name="add_x_and_y", x=1, y=2)
wg.add_task(
    add_x_and_y_and_z, 
    name="add_x_and_y_and_z", 
    z=wg.tasks["add_x_and_y"].outputs["z"], 
    x=wg.tasks["add_x_and_y"].outputs["x"], 
    y=wg.tasks["add_x_and_y"].outputs["y"],
)

Widget dependency not found. To visualize the workgraph, please install the widget dependency. Use 'pip install aiida-workgraph[widget]' if installing from PyPI. For local source installations, use 'pip install .[widget]' and then build the JavaScript library. Refer to the documentation for more details.


DecoratedNode(name='add_x_and_y_and_z', properties=[], inputs=['metadata', 'metadata.store_provenance', 'metadata.description', 'metadata.label', 'metadata.call_link_label', 'metadata.disable_cache', 'x', 'y', 'z', '_wait'], outputs=['result', '_outputs', '_wait'])

In [7]:
work_graph_dict = wg.to_dict()

In [8]:
edges_label_lst = []
for link_dict in work_graph_dict["links"]:
    if link_dict['from_socket'] == "result":
        edges_label_lst.append({'target': link_dict['to_node'], 'targetHandle': link_dict['to_socket'], 'source': link_dict['from_node'], 'sourceHandle': None})
    else:
        edges_label_lst.append({'target': link_dict['to_node'], 'targetHandle': link_dict['to_socket'], 'source': link_dict['from_node'], 'sourceHandle': link_dict['from_socket']})

edges_label_lst

[{'target': 'add_x_and_y_and_z',
  'targetHandle': 'z',
  'source': 'add_x_and_y',
  'sourceHandle': 'z'},
 {'target': 'add_x_and_y_and_z',
  'targetHandle': 'x',
  'source': 'add_x_and_y',
  'sourceHandle': 'x'},
 {'target': 'add_x_and_y_and_z',
  'targetHandle': 'y',
  'source': 'add_x_and_y',
  'sourceHandle': 'y'}]

In [9]:
kwargs_dict, function_dict = {}, {}
for task_name, task_dict in work_graph_dict["tasks"].items():
    input_variables = [
        input_parameter 
        for input_parameter in task_dict['inputs'].keys() 
        if not input_parameter.startswith("metadata") and not input_parameter.startswith("_wait")
    ]
    input_kwargs = {
        input_parameter: task_dict['inputs'][input_parameter]['property']["value"].value if isinstance(task_dict['inputs'][input_parameter]['property']["value"], dict) else task_dict['inputs'][input_parameter]['property']["value"] 
        for input_parameter in input_variables
    }
    kwargs_dict[task_name] = input_kwargs
    function_dict[task_name] = loads(task_dict['executor']['executor']).process_class._func
kwargs_dict, function_dict

({'add_x_and_y': {'x': 1, 'y': 2},
  'add_x_and_y_and_z': {'x': None, 'y': None, 'z': None}},
 {'add_x_and_y': <function __main__.add_x_and_y(x, y)>,
  'add_x_and_y_and_z': <function __main__.add_x_and_y_and_z(x, y, z)>})

In [10]:
nodes_dict, mapping_dict = {}, {}
for i, [k, v] in enumerate(function_dict.items()):
    nodes_dict[i] = v
    mapping_dict[k] = i

nodes_dict, mapping_dict 

({0: <function __main__.add_x_and_y(x, y)>,
  1: <function __main__.add_x_and_y_and_z(x, y, z)>},
 {'add_x_and_y': 0, 'add_x_and_y_and_z': 1})

In [11]:
value_dict = {}
for func_name, val_dict in kwargs_dict.items():
    for k, v in val_dict.items():
        if v is not None:
            if func_name not in value_dict.keys():
                value_dict[func_name] = {}
            value_dict[func_name][k] = v

In [12]:
i = len(nodes_dict)
for val_dict in value_dict.values():
    for k, v in val_dict.items():
        nodes_dict[i] = v
        mapping_dict[v] = i
        i += 1

nodes_dict, mapping_dict

({0: <function __main__.add_x_and_y(x, y)>,
  1: <function __main__.add_x_and_y_and_z(x, y, z)>,
  2: 1,
  3: 2},
 {'add_x_and_y': 0, 'add_x_and_y_and_z': 1, 1: 2, 2: 3})

In [13]:
for func_name, val_dict in kwargs_dict.items():
    for k, v in val_dict.items():
        if v is not None:
            edges_label_lst.append({'target': func_name, 'targetHandle': k, 'source': v, 'sourceHandle': None})
edges_label_lst

[{'target': 'add_x_and_y_and_z',
  'targetHandle': 'z',
  'source': 'add_x_and_y',
  'sourceHandle': 'z'},
 {'target': 'add_x_and_y_and_z',
  'targetHandle': 'x',
  'source': 'add_x_and_y',
  'sourceHandle': 'x'},
 {'target': 'add_x_and_y_and_z',
  'targetHandle': 'y',
  'source': 'add_x_and_y',
  'sourceHandle': 'y'},
 {'target': 'add_x_and_y',
  'targetHandle': 'x',
  'source': 1,
  'sourceHandle': None},
 {'target': 'add_x_and_y',
  'targetHandle': 'y',
  'source': 2,
  'sourceHandle': None}]

In [14]:
edges_lst = []
for edge in edges_label_lst:
    edges_lst.append({'target': mapping_dict[edge['target']], 'targetHandle': edge['targetHandle'], 'source': mapping_dict[edge['source']], 'sourceHandle': edge['sourceHandle']})

edges_lst

[{'target': 1, 'targetHandle': 'z', 'source': 0, 'sourceHandle': 'z'},
 {'target': 1, 'targetHandle': 'x', 'source': 0, 'sourceHandle': 'x'},
 {'target': 1, 'targetHandle': 'y', 'source': 0, 'sourceHandle': 'y'},
 {'target': 0, 'targetHandle': 'x', 'source': 2, 'sourceHandle': None},
 {'target': 0, 'targetHandle': 'y', 'source': 3, 'sourceHandle': None}]

In [15]:
nodes_dict

{0: <function __main__.add_x_and_y(x, y)>,
 1: <function __main__.add_x_and_y_and_z(x, y, z)>,
 2: 1,
 3: 2}